In [1]:
!python tools/create_train_data_from_label_xml.py

data/train_bounding/049.png
data/train_bounding/000.png
data/train_bounding/132.png
data/train_bounding/017.png
data/train_bounding/023.png
data/train_bounding/031.png
data/train_bounding/127.png
data/train_bounding/030.png
data/train_bounding/037.png
data/train_bounding/015.png
data/train_bounding/022.png
data/train_bounding/028.png
data/train_bounding/038.png
data/train_bounding/010.png
data/train_bounding/041.png
data/train_bounding/014.png
data/train_bounding/018.png
data/train_bounding/032.png
data/train_bounding/051.png
data/train_bounding/008.png
data/train_bounding/130.png
data/train_bounding/126.png
data/train_bounding/043.png
data/train_bounding/034.png
data/train_bounding/024.png
data/train_bounding/019.png
data/train_bounding/096.png
data/train_bounding/003.png
data/train_bounding/001.png
data/train_bounding/128.png
data/train_bounding/046.png
data/train_bounding/012.png
data/train_bounding/134.png
data/train_bounding/045.png
data/train_bounding/029.png
data/train_bounding/

In [12]:
!git pull

remote: Counting objects: 4, done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/vnbot2/thesis
   d0ecbcb..bde2657  master     -> origin/master
Updating d0ecbcb..bde2657
Fast-forward
 tools/create_data.py | 3 +--
 1 file changed, 1 insertion(+), 2 deletions(-)


In [16]:
from glob import glob
len(glob('data/synthetic_data_bounding/*.png'))

32000

In [15]:
!ls data

__MACOSX		 text_line_verified	 train_bounding
synthetic_data_bounding  text_line_verified.zip


In [1]:
! ls pix2pix/output/

128x1024_64_data


In [2]:
!bash script/pix2pix_train

loaded lab_colorization = False
loaded ndf = 32
loaded ngf = 32
loaded which_direction = AtoB
aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = pix2pix/output/128x1024_64_data
display_freq = 0
flip = False
gan_weight = 1.0
input_dir = None
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = None
max_steps = None
mode = export
ndf = 32
ngf = 32
output_dir = pix2pix/frozen/128x1024
output_filetype = png
progress_freq = 50
save_freq = 1000
scale_size = [128, 1024]
seed = 2139197480
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
Batch input: Tensor("truediv:0", shape=(?, 128, 1024, 3), dtype=float32)
Create GENERATOR--------------------------------
(?, 64, 512, 32)
(?, 32, 256, 64)
(?, 16, 128, 128)
(?, 8, 64, 256)
(?, 4, 32, 256)
(?, 2, 16, 256)
(?, 1, 8, 256)
(?, 1, 4, 256)
DECONV
(?, 1, 8, 256)
(?, 2, 16, 256)
(?, 4, 32, 256)
(?, 8, 64, 256)
(?, 16, 128, 128)
(?, 32, 256, 64)
(?, 64, 512, 32)
(?, 128, 1024, 3)
image: Tensor("gene

In [120]:
from time import time
import tensorflow as tf
import cv2
import numpy as np
import os
from glob import glob
import argparse
import math

class argument:
    def __init__(self):
        self.strides = [64, 256]
        self.k_size =[128,512]
        self.checkpoint = 'pix2pix/output/frozen/128x1024_v3/'
        self.input_dir = 'data/ano'
        self.output_dir = 'output/run_method2'
args = argument()
meta_path = '{}/export.meta'.format(args.checkpoint)
print('meta path:', meta_path)
assert os.path.exists(meta_path)
tf.reset_default_graph()
tf.train.import_meta_graph(meta_path)
sess = tf.Session()
saver = tf.train.Saver()

saver.restore(sess, tf.train.latest_checkpoint(
    args.checkpoint))





def get_tensor_by_name(name):
    name_on_device = '{}:0'.format(name)
    return tf.get_default_graph().get_tensor_by_name(name_on_device)


def resize(image):
    h, w = image.shape[:2]
    new_h = math.ceil(h/args.strides[0])*args.strides[0]
    new_w = math.ceil(w/args.strides[1])*args.strides[1]
    return cv2.resize(image, (new_w, new_h))

def extract_patches(image, k_size, strides):
    images = tf.extract_image_patches(tf.expand_dims(
        image, 0), k_size, strides, rates=[1, 1, 1, 1], padding='SAME')[0]
    images_shape = tf.shape(images)
    images_reshape = tf.reshape(
        images, [images_shape[0]*images_shape[1], *k_size[1:3], 3])
    images, n1, n2 = tf.cast(images_reshape, tf.uint8) , images_shape[0], images_shape[1]
    return images, n1, n2

def join_patches(images, n1, n2, k_size, strides):

    s1 = k_size[1]//2-strides[1]//2
    s2 = k_size[2]//2-strides[2]//2
    roi = images[:, 
                 s1:s1+strides[1],\
                 s2:s2+strides[2],
                 :]
    new_shape = [n1, n2, *roi.shape[1:]]
    reshaped_roi = tf.reshape(roi, new_shape)
    reshaped_roi = tf.transpose(reshaped_roi, perm=[0,2,1,3,4])
    rs = tf.shape(reshaped_roi)
    rv = tf.cast(tf.reshape(reshaped_roi, [rs[0]*rs[1], rs[2]*rs[3], -1]), tf.uint8)
    return rv

def run_image(image):
    h, w = image.shape[:2]
    resized_image = resize(image)
    splited_images, n1, n2 = extract_patches(image, [1, 128, 512,1], [1,64,256,1])
    print('n1,n2:',sess.run([n1, n2]))
    splited_images_ = sess.run(splited_images)
    print(type(splited_images_), splited_images_.shape)
    output_images = sess.run(outputs, feed_dict={inputs: splited_images_})
    
    output_image = join_patches(output_images, n1, n2, [1, 128, 512,1], [1,64,256,1])
    output_image = output_image[args.k_size[0]:-args.k_size[0],  #channel1
                                args.k_size[1]:-args.k_size[1],  #channel2
                                :]#channel 3
    output_image = sess.run(output_image)
    output_image = cv2.resize(output_image, (w, h))

    return output_image

meta path: pix2pix/output/frozen/128x1024_v3//export.meta
INFO:tensorflow:Restoring parameters from pix2pix/output/frozen/128x1024_v3/export


In [121]:
!rm -r output/run_method3
args.output_dir = 'output/run_method3'

In [124]:
os.makedirs(args.output_dir, exist_ok=True)
start = time()
paths = glob('{}/*.png'.format(args.input_dir))
paths = [path for path in paths]
assert len(paths) > 0
print('Num of sample:', len(paths), args.input_dir)
inputs = get_tensor_by_name('inputs')
outputs = get_tensor_by_name('outputs')

for path in paths:
    print(path)
    name = path.split('/')[-1].split('.')[0]
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output_image = sess.run(outputs, {inputs:image})
    merge_image = 0.5*image+0.5*output_image
    # os.makedirs('output/{}_{}'.format(args.stride, name), exist_ok=True)
    print('{}/{}_{}_input.png'.format(args.output_dir,args.strides, name))
    cv2.imwrite('{}/{}_{}_input.png'.format(args.output_dir,args.strides, name), image)
    cv2.imwrite('{}/{}_{}_output.png'.format(args.output_dir, args.strides, name), output_image)
    cv2.imwrite('{}/{}_{}_merge.png'.format(args.output_dir, args.strides, name), merge_image)
print('Running time:', time()-start)

Num of sample: 151 data/ano
data/ano/77.png
output/run_method3/[64, 256]_77_input.png
data/ano/63.png
output/run_method3/[64, 256]_63_input.png
data/ano/62.png
output/run_method3/[64, 256]_62_input.png
data/ano/76.png
output/run_method3/[64, 256]_76_input.png
data/ano/89.png
output/run_method3/[64, 256]_89_input.png
data/ano/149.png
output/run_method3/[64, 256]_149_input.png
data/ano/60.png
output/run_method3/[64, 256]_60_input.png
data/ano/74.png
output/run_method3/[64, 256]_74_input.png
data/ano/48.png
output/run_method3/[64, 256]_48_input.png
data/ano/49.png
output/run_method3/[64, 256]_49_input.png
data/ano/75.png
output/run_method3/[64, 256]_75_input.png
data/ano/61.png
output/run_method3/[64, 256]_61_input.png
data/ano/148.png
output/run_method3/[64, 256]_148_input.png
data/ano/59.png
output/run_method3/[64, 256]_59_input.png
data/ano/65.png
output/run_method3/[64, 256]_65_input.png
data/ano/70.png
output/run_method3/[64, 256]_70_input.png
data/ano/64.png
output/run_method3/[64, 

KeyboardInterrupt: 

In [74]:
!rm -r output/run_method3/

In [43]:
args.output_dir

'output/run_method2'

In [1]:
import tensorflow as tf


/Users/macos/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [21]:
tf.reset_default_graph()
a = tf.placeholder(tf.float32, [None, 100, 100, 3])
b = tf.layers.conv2d(a, 1, 3)
row = tf.train.slice_input_producer([a], num_epochs=1, shuffle=False)[0]

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,100,100,3]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,100,100,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]


In [10]:
tf.shape(a)[-1]

<tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>